In [15]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import time

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE

In [16]:

def selectKbest(indep,dep,n):
    test=SelectKBest(score_func=chi2, k=n)
    #fit1=test.fit(indep,dep)
    selectk_features=test.fit_transform(indep,dep)
    return selectk_features

def Split_Scaler(indep,dep):
    x_train,x_test,y_train,y_test=train_test_split(indep,dep,test_size=0.30,random_state=0)
    scX=StandardScaler()
    x_train=scX.fit_transform(x_train)
    x_test=scX.transform(x_test)
    return x_train,x_test,y_train,y_test

def cm_predition(classifier, x_test):
    y_pred=classifier.predict(x_test)

    #making the Confusion Matrix
    from sklearn.metrics import confusion_matrix
    cm=confusion_matrix(y_test,y_pred)

    # making Classification Report
    from sklearn.metrics import classification_report
    clf_report=classification_report(y_test,y_pred)

    # finding the accuracy score
    from sklearn.metrics import accuracy_score
    accuracy_score=accuracy_score(y_test,y_pred)
    
    return classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred

def logistic(x_train,y_train,x_test):
    from sklearn.linear_model import LogisticRegression
    classifier=LogisticRegression(random_state=0)
    classifier.fit(x_train,y_train)
    classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred = cm_predition(classifier,x_test)
    return classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred 
    
def svm_linear(x_train,y_train,x_test):
    from sklearn.svm import SVC
    classifier=SVC(kernel='linear',random_state=0)
    classifier.fit(x_train,y_train)
    classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred = cm_predition(classifier,x_test)
    return classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred 
    
def svm_nonlinear(x_train,y_train,x_test):
    from sklearn.svm import SVC
    classifier=SVC(kernel='rbf', random_state=0)
    classifier.fit(x_train,y_train)
    classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred = cm_predition(classifier,x_test)
    return classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred 
    
def random(x_train,y_train,x_test):
    from sklearn.ensemble import RandomForestClassifier
    classifier=RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
    classifier.fit(x_train,y_train)
    classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred = cm_predition(classifier,x_test)
    return classifier,cm,clf_report,accuracy_score,x_test,y_test ,y_pred 

def Decision(x_train,y_train,x_test):
    from sklearn.tree import DecisionTreeClassifier
    classifier=DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
    classifier.fit(x_train,y_train)
    classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred = cm_predition(classifier,x_test)
    return classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred 

def knn(x_train,y_train,x_test):
    from sklearn.neighbors import KNeighborsClassifier
    classifier=KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
    classifier.fit(x_train,y_train)
    classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred = cm_predition(classifier,x_test)
    return classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred 

def navie(x_train,y_train,x_test):
    from sklearn.naive_bayes import GaussianNB
    classifier=GaussianNB()
    classifier.fit(x_train,y_train)
    classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred = cm_predition(classifier,x_test)
    return classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred 

# Displaying the values in the tabular frame

def selectK_classification(acc_logistic, acc_svmL, acc_svmNL, acc_knn, acc_rf, acc_des, acc_nav):
    
    table=pd.DataFrame(index=["ChiSquare"],columns=["Logistic","SVM_linear","SVM_Nonlinear","KNN","Random","Decision","Navie"])
    for number,idex in enumerate(table.index):
         table['Logistic'][idex]= acc_logistic[number]
         table['SVM_linear'][idex]= acc_svmL[number]
         table['SVM_Nonlinear'][idex]= acc_svmNL[number]
         table['KNN'][idex]= acc_knn[number]
         table['Random'][idex]= acc_rf[number]
         table['Decision'][idex]= acc_des[number]
         table['Navie'][idex]= acc_nav[number]
         
    return table


def selectk_Classification(acclog, accsvml, accsvmnl, accknn, accnav, accdes, accrf): 
    dataframe = pd.DataFrame(index=['ChiSquare'], columns=['Logistic', 'SVMl', 'SVMnl', 'KNN', 'Navie', 'Decision', 'Random'])
    for number, idex in enumerate(dataframe.index):      
        dataframe['Logistic'][idex] = acclog[number]       
        dataframe['SVMl'][idex] = accsvml[number]
        dataframe['SVMnl'][idex] = accsvmnl[number]
        dataframe['KNN'][idex] = accknn[number]
        dataframe['Navie'][idex] = accnav[number]
        dataframe['Decision'][idex] = accdes[number]
        dataframe['Random'][idex] = accrf[number]
    return dataframe

In [29]:
dataset=pd.read_csv("prep.csv",index_col=None)
df1=dataset  

#Convert categorical variable into dummy/indicator variables.
df1=pd.get_dummies(df1,drop_first=True,dtype=int)
df1

# separation of independent and dependent variables
indep=df1.drop("classification_yes",axis=1)
dep=df1["classification_yes"]

indep
dep   
    
kbest=selectKbest(indep, dep, 1) 
#kbest

acc_logistic=[]
acc_svmL=[]
acc_svmNL=[]
acc_knn=[]
acc_nav=[]
acc_des=[]
acc_rf=[]

kbest

array([[ 8408.19112628],
       [12300.        ],
       [ 8408.19112628],
       [ 8408.19112628],
       [12400.        ],
       [16700.        ],
       [ 6000.        ],
       [ 6700.        ],
       [ 8408.19112628],
       [ 8408.19112628],
       [ 8600.        ],
       [10300.        ],
       [ 6600.        ],
       [ 9500.        ],
       [ 7700.        ],
       [ 6200.        ],
       [ 7000.        ],
       [11200.        ],
       [ 7200.        ],
       [ 6900.        ],
       [ 8408.19112628],
       [ 4500.        ],
       [ 5500.        ],
       [ 8408.19112628],
       [12400.        ],
       [ 7900.        ],
       [ 9800.        ],
       [ 7200.        ],
       [ 4300.        ],
       [ 6300.        ],
       [ 6900.        ],
       [ 5800.        ],
       [ 5600.        ],
       [10700.        ],
       [ 6900.        ],
       [ 9200.        ],
       [ 7000.        ],
       [ 6900.        ],
       [ 8408.19112628],
       [ 8600.        ],


In [30]:
x_train,x_test,y_train,y_test = Split_Scaler(kbest, dep) 

classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred = logistic(x_train,y_train,x_test)
acc_logistic.append(accuracy_score)

classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred = svm_linear(x_train,y_train,x_test)
acc_svmL.append(accuracy_score)

classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred = svm_nonlinear(x_train,y_train,x_test)
acc_svmNL.append(accuracy_score)

classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred = knn(x_train,y_train,x_test)
acc_knn.append(accuracy_score)

classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred= navie(x_train,y_train,x_test)
acc_nav.append(accuracy_score)

classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred = Decision(x_train,y_train,x_test)
acc_des.append(accuracy_score)

classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred = random(x_train,y_train,x_test)
acc_rf.append(accuracy_score)

result= selectK_classification(acc_logistic,acc_svmL,acc_svmNL,acc_knn,acc_rf,acc_des,acc_nav)


/opt/anaconda3/envs/venvaiml/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/venvaiml/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/venvaiml/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu

In [19]:
result #5

,Logistic,SVM_linear,SVM_Nonlinear,KNN,Random,Decision,Navie
ChiSquare,0.941667,0.941667,0.95,0.908333,0.958333,0.958333,0.841667


In [22]:
result #4

,Logistic,SVM_linear,SVM_Nonlinear,KNN,Random,Decision,Navie
ChiSquare,0.858333,0.841667,0.833333,0.858333,0.908333,0.891667,0.791667


In [25]:
result #3

,Logistic,SVM_linear,SVM_Nonlinear,KNN,Random,Decision,Navie
ChiSquare,0.791667,0.808333,0.825,0.833333,0.85,0.883333,0.783333


In [28]:
result #2

,Logistic,SVM_linear,SVM_Nonlinear,KNN,Random,Decision,Navie
ChiSquare,0.708333,0.675,0.691667,0.791667,0.733333,0.691667,0.716667


In [31]:
result #1

,Logistic,SVM_linear,SVM_Nonlinear,KNN,Random,Decision,Navie
ChiSquare,0.641667,0.625,0.633333,0.666667,0.641667,0.608333,0.625


In [ ]:
# Feature set 5 and Random or Decision tree algs combination have the ChiSqueare value high.....